In [1]:
%load_ext watermark
%watermark -a Chan -d -p numpy,pandas,konlpy,torch,keras

Using TensorFlow backend.


Chan 2019-08-23 

numpy 1.16.4
pandas 0.24.2
konlpy 0.5.1
torch 1.0.1
keras 2.2.4


In [2]:
import pandas as pd
import numpy as np
import konlpy
from utils.bp_processing import bp_tokenize

# Load datasets 

In [3]:
datasets = pd.read_csv('../../data/auto_labeled.csv', encoding='utf-16', sep='\t')
datasets.head()
tmp = datasets.label

In [4]:
datasets.dropna(inplace=True)

In [5]:
datasets.label = datasets.label.astype('int32')

In [6]:
datasets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1999993 entries, 0 to 1999993
Data columns (total 2 columns):
comment    object
label      int32
dtypes: int32(1), object(1)
memory usage: 38.1+ MB


In [7]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from imblearn.under_sampling import RandomUnderSampler
import torch.utils.data as data_utils
import torch

In [8]:
tmp = [[_] for _ in datasets.comment]
label = datasets.label 

In [9]:
del datasets

In [10]:
# undersampling
X, y = RandomUnderSampler().fit_sample(tmp, label)
len(y), sum(y)

KeyboardInterrupt: 

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
X_train = [_[0] for _ in X_train]
X_test = [_[0] for _ in X_test]

In [ ]:
# bpe tokenizing
tokenized_train = bp_tokenize(X_train)
tokenized_test = bp_tokenize(X_test)

In [ ]:
tokenized_train.shape, tokenized_test.shape

In [ ]:
# padding
MAX_LEN = 30
x_padded_train = pad_sequences(tokenized_train, maxlen=MAX_LEN)
x_padded_test = pad_sequences(tokenized_test, maxlen=MAX_LEN)

In [ ]:
train_data = data_utils.TensorDataset(torch.from_numpy(x_padded_train).type(torch.LongTensor),torch.from_numpy(y_train).type(torch.DoubleTensor))
BATCH_SIZE = 512
train_loader = data_utils.DataLoader(train_data, batch_size=BATCH_SIZE, drop_last=True)
# return train_loader,x_test_pad,y_test

# Model

In [ ]:
import torch,keras
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as data_utils
 
EMB_DIM = 10
VOCAB_SIZE = 260
class StructuredSelfAttention(torch.nn.Module):
    """
    The class is an implementation of the paper A Structured Self-Attentive Sentence Embedding including regularization
    and without pruning. Slight modifications have been done for speedup
    """
   
    def __init__(self, batch_size, lstm_hid_dim, d_a, r, max_len, emb_dim=EMB_DIM, vocab_size=VOCAB_SIZE):
        """
        Initializes parameters suggested in paper
 
        Args:
            batch_size  : {int} batch_size used for training
            lstm_hid_dim: {int} hidden dimension for lstm
            d_a         : {int} hidden dimension for the dense layer
            r           : {int} attention-hops or attention heads
            max_len     : {int} number of lstm timesteps
            emb_dim     : {int} embeddings dimension
            vocab_size  : {int} size of the vocabulary
 
        Returns:
            self
 
        Raises:
            Exception
        """
        super(StructuredSelfAttention,self).__init__()
       
        self.embeddings = torch.nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.lstm = torch.nn.LSTM(emb_dim, lstm_hid_dim, 1, batch_first=True)
        self.linear_first = torch.nn.Linear(lstm_hid_dim, d_a)
        self.linear_first.bias.data.fill_(0)
        self.linear_second = torch.nn.Linear(d_a, r)
        self.linear_second.bias.data.fill_(0)
        self.n_classes = 1
        self.linear_final = torch.nn.Linear(lstm_hid_dim, self.n_classes)
        self.batch_size = batch_size       
        self.max_len = max_len
        self.lstm_hid_dim = lstm_hid_dim
        self.hidden_state = self.init_hidden()
        self.r = r
        
    def softmax(self,input, axis=1):
        """
        Softmax applied to axis=n
 
        Args:
           input: {Tensor,Variable} input on which softmax is to be applied
           axis : {int} axis on which softmax is to be applied
 
        Returns:
            softmaxed tensors
 
       
        """
 
        input_size = input.size()
        trans_input = input.transpose(axis, len(input_size)-1)
        trans_size = trans_input.size()
        input_2d = trans_input.contiguous().view(-1, trans_size[-1])
        soft_max_2d = F.softmax(input_2d)
        soft_max_nd = soft_max_2d.view(*trans_size)
        return soft_max_nd.transpose(axis, len(input_size)-1)
       
        
    def init_hidden(self):
        return (Variable(torch.zeros(1,self.batch_size,self.lstm_hid_dim)),Variable(torch.zeros(1,self.batch_size,self.lstm_hid_dim)))
       
        
    def forward(self,x):
        embeddings = self.embeddings(x)
        outputs, self.hidden_state = self.lstm(embeddings.view(self.batch_size,self.max_len,-1),self.hidden_state)       
        x = F.tanh(self.linear_first(outputs))       
        x = self.linear_second(x)       
        x = self.softmax(x,1)       
        attention = x.transpose(1,2)       
        sentence_embeddings = attention@outputs       
        avg_sentence_embeddings = torch.sum(sentence_embeddings,1)/self.r
        
        output = F.sigmoid(self.linear_final(avg_sentence_embeddings))
        return output,attention

       
    #Regularization
    def l2_matrix_norm(self,m):
        """
        Frobenius norm calculation
 
        Args:
           m: {Variable} ||AAT - I||
 
        Returns:
            regularized value
 
       
        """
        return torch.sum(torch.sum(torch.sum(m**2,1),1)**0.5).type(torch.DoubleTensor)

# Train

In [ ]:
import torch
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, f1_score, classification_report

def train(attention_model, train_loader, criterion, optimizer, epochs = 5, use_regularization = False , C=0, clip=False):
    """
        Training code
 
        Args:
            attention_model : {object} model
            train_loader    : {DataLoader} training data loaded into a dataloader
            optimizer       :  optimizer
            criterion       :  loss function. Must be BCELoss for binary_classification and NLLLoss for multiclass
            epochs          : {int} number of epochs
            use_regularizer : {bool} use penalization or not
            C               : {int} penalization coeff
            clip            : {bool} use gradient clipping or not
       
        Returns:
            accuracy and losses of the model
 
      
        """
    losses = []
    accuracy = []
    for i in range(epochs):
        print("Running EPOCH",i+1)
        total_loss = 0
        n_batches = 0
        correct = 0
       
        for batch_idx,train in enumerate(train_loader):
 
            attention_model.hidden_state = attention_model.init_hidden()
            x,y = Variable(train[0]),Variable(train[1])
            y_pred,att = attention_model(x)
           
            #penalization AAT - I
            if use_regularization:
                attT = att.transpose(1,2)
                identity = torch.eye(att.size(1))
                identity = Variable(identity.unsqueeze(0).expand(train_loader.batch_size,att.size(1),att.size(1)))
                penal = attention_model.l2_matrix_norm(att@attT - identity)

            #binary classification
            #Adding a very small value to prevent BCELoss from outputting NaN's
            correct+=torch.eq(torch.round(y_pred.type(torch.DoubleTensor).squeeze(1)),y).data.sum()
            if use_regularization:
                try:
                    loss = criterion(y_pred.type(torch.DoubleTensor).squeeze(1)+1e-8,y) + C * penal/train_loader.batch_size

                except RuntimeError:
                    raise Exception("BCELoss gets nan values on regularization. Either remove regularization or add very small values")
            else:
                loss = criterion(y_pred.type(torch.DoubleTensor).squeeze(1),y)
               
 
            total_loss+=loss.data
            optimizer.zero_grad()
            loss.backward()
           
            #gradient clipping
            if clip:
                torch.nn.utils.clip_grad_norm(attention_model.parameters(),0.5)
            optimizer.step()
            n_batches+=1
           
        print("avg_loss is",total_loss/n_batches)
        print("Accuracy of the model",int(correct)/(n_batches*train_loader.batch_size))
        losses.append(total_loss/n_batches)
        accuracy.append(int(correct)/(n_batches*train_loader.batch_size))
        
        
    return losses, accuracy
 
def evaluate(attention_model, x_test, y_test):
    """
        cv results
 
        Args:
            attention_model : {object} model
            x_test          : {nplist} x_test
            y_test          : {nplist} y_test
       
        Returns:
            cv-accuracy
 
      
    """
   
    attention_model.batch_size = x_test.shape[0]
    attention_model.hidden_state = attention_model.init_hidden()
    x_test_var = Variable(torch.from_numpy(x_test).type(torch.LongTensor))
    y_test_pred,_ = attention_model(x_test_var)
    
    y_preds = torch.round(y_test_pred.type(torch.DoubleTensor).squeeze(1))
    y_test_var = Variable(torch.from_numpy(y_test).type(torch.DoubleTensor))
    
    print(classification_report(y_test, y_preds.detach().numpy()))
    return int(torch.eq(y_preds,y_test_var).data.sum())/x_test_var.size(0)
 
def get_activation_wts(attention_model,x):
    """
        Get r attention heads
 
        Args:
            attention_model : {object} model
            x               : {torch.Variable} input whose weights we want
       
        Returns:
            r different attention weights
 
      
    """
    attention_model.batch_size = x.size(0)
    attention_model.hidden_state = attention_model.init_hidden()
    _,wts = attention_model(x)
    return wts

In [ ]:
attention_model = StructuredSelfAttention(batch_size=train_loader.batch_size,
                                          lstm_hid_dim=10,
                                          d_a=100,
                                          r=1,
                                          vocab_size=VOCAB_SIZE, 
                                          max_len=MAX_LEN,
                                          )

In [43]:
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(attention_model.parameters())

losses, accuracy = train(attention_model, train_loader, loss, optimizer, epochs=15,
                         use_regularization=False, C=0.03, clip=True)


Running EPOCH 1


C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\ipykernel_launcher.py:64: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


avg_loss is tensor(0.6550, dtype=torch.float64)
Accuracy of the model 0.6098769203910615
Running EPOCH 2
avg_loss is tensor(0.5672, dtype=torch.float64)
Accuracy of the model 0.7135562150837989
Running EPOCH 3
avg_loss is tensor(0.4954, dtype=torch.float64)
Accuracy of the model 0.7690729748603352
Running EPOCH 4
avg_loss is tensor(0.4381, dtype=torch.float64)
Accuracy of the model 0.800039280726257
Running EPOCH 5
avg_loss is tensor(0.3932, dtype=torch.float64)
Accuracy of the model 0.8332314944134078
Running EPOCH 6
avg_loss is tensor(0.3570, dtype=torch.float64)
Accuracy of the model 0.8499367143854749
Running EPOCH 7
avg_loss is tensor(0.3306, dtype=torch.float64)
Accuracy of the model 0.8625174581005587
Running EPOCH 8
avg_loss is tensor(0.3113, dtype=torch.float64)
Accuracy of the model 0.8710064594972067
Running EPOCH 9
avg_loss is tensor(0.2962, dtype=torch.float64)
Accuracy of the model 0.8781424581005587
Running EPOCH 10
avg_loss is tensor(0.2836, dtype=torch.float64)
Accurac

# Evaluate 

In [44]:
print(evaluate(attention_model, x_padded_test, y_test))

C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


              precision    recall  f1-score   support

           0       0.88      0.94      0.91     15279
           1       0.93      0.87      0.90     15290

    accuracy                           0.90     30569
   macro avg       0.91      0.90      0.90     30569
weighted avg       0.91      0.90      0.90     30569

0.9030717393437796


# Visualize

In [51]:
def visualize_attention(wts, X_test, filename):
    wts_add = torch.sum(wts, 1)
    wts_add_np = wts_add.data.numpy()
    wts_add_list = wts_add_np.tolist()
    text = []
    for test in X_test:
        text.append(" ".join(test))
    createHTML(text, wts_add_list, filename)
    print("Attention visualization created for {} samples".format(len(X_test)))
    return

In [52]:
attention_model.batch_size = np.array(x_padded_test).shape[0]
attention_model.hidden_state = attention_model.init_hidden()
x_test_var = Variable(torch.from_numpy(x_padded_test).type(torch.LongTensor))
y_test_pred,_ = attention_model(x_test_var)
y_preds = torch.round(y_test_pred.type(torch.DoubleTensor).squeeze(1))

C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [53]:
test_last_idx = -500
wts = get_activation_wts(attention_model,
                         Variable(torch.from_numpy(x_padded_test[y_preds.data.numpy()==1][test_last_idx:]).type(torch.LongTensor)))
# print(torch.sum(wts,1).data.numpy().tolist())
visualize_attention(wts, np.array(X_test)[y_preds.data.numpy()==1][test_last_idx:], filename='bpe_attention.html')

C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Attention visualization created for 500 samples


In [50]:
#Credits to Lin Zhouhan(@hantek) for the complete visualization code
import random, os, numpy, scipy
from codecs import open
def createHTML(texts, weights, fileName):
    """
    Creates a html file with text heat.
	weights: attention weights for visualizing
	texts: text on which attention weights are to be visualized
    """
    fileName = "visualization/"+fileName
    fOut = open(fileName, "w", encoding="utf-8")
    part1 = """
    <html lang="en">
    <head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8">
    <style>
    body {
    font-family: Sans-Serif;
    }
    </style>
    </head>
    <body>
    <h3>
    Heatmaps
    </h3>
    </body>
    <script>
    """
    part2 = """
    var color = "255,0,0";
    var ngram_length = 1;
    var half_ngram = 3;
    for (var k=0; k < any_text.length; k++) {
    var tokens = any_text[k].split(" ");
    var intensity = new Array(tokens.length);
    var max_intensity = Number.MIN_SAFE_INTEGER;
    var min_intensity = Number.MAX_SAFE_INTEGER;
    for (var i = 0; i < intensity.length; i++) {
    intensity[i] = 0.0;
    for (var j = -half_ngram; j < ngram_length-half_ngram; j++) {
    if (i+j < intensity.length && i+j > -1) {
    intensity[i] += trigram_weights[k][i + j];
    }
    }
    if (i == 0 || i == intensity.length-1) {
    intensity[i] /= 2.0;
    } else {
    intensity[i] /= 3.0;
    }
    if (intensity[i] > max_intensity) {
    max_intensity = intensity[i];
    }
    if (intensity[i] < min_intensity) {
    min_intensity = intensity[i];
    }
    }
    var denominator = max_intensity - min_intensity;
    for (var i = 0; i < intensity.length; i++) {
    intensity[i] = (intensity[i] - min_intensity) / denominator;
    }
    if (k%2 == 0) {
    var heat_text = "<p><br><b>Example:</b><br>";
    } else {
    var heat_text = "<b>Example:</b><br>";
    }
    var space = "";
    for (var i = 0; i < tokens.length; i++) {
    heat_text += "<span style='background-color:rgba(" + color + "," + intensity[i] + ")'>" + space + tokens[i] + "</span>";
    if (space == "") {
    space = " ";
    }
    }
    //heat_text += "<p>";
    document.body.innerHTML += heat_text;
    }
    </script>
    </html>"""
    putQuote = lambda x: "\"%s\""%x
    textsString = "var any_text = [%s];\n"%(",".join(map(putQuote, texts)))
    weightsString = "var trigram_weights = [%s];\n"%(",".join(map(str,weights)))
    fOut.write(part1)
    fOut.write(textsString)
    fOut.write(weightsString)
    fOut.write(part2)
    fOut.close()
  
    return


# Model Save

In [54]:
# torch.save(attention_model, '../model/self_attention_bp.pt')

C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\torch\serialization.py:251: UserWarning: Couldn't retrieve source code for container of type StructuredSelfAttention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
